In [1]:
import pandas as pd
import geopandas as gpd

### Import all bridge projects in Allegheny County

In [2]:
future = gpd.read_file('input/Future_Development___All.zip')
future = future.set_crs('EPSG:3857')

indev = gpd.read_file('input/In_Development___All.zip')
indev = indev.set_crs('EPSG:3857')

construct = gpd.read_file('input/Under_Construction___All.zip')
construct = construct.set_crs('EPSG:3857')

In [3]:
df = pd.concat([future, indev, construct])
df = df.to_crs('EPSG:4326')

In [4]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1041 entries, 0 to 467
Columns: 125 entries, GISDATA_PR to geometry
dtypes: float64(11), geometry(1), int64(13), object(100)
memory usage: 1.0+ MB


### Restrict to Pittsburgh-owned bridges, only include projects once

In [5]:
df = df[df['PROJECT_SP'] == 'City of Pittsburgh']

In [6]:
df = df.drop_duplicates('PROJECT_ID')
df = df.reset_index(drop=True)

In [7]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 24 entries, 0 to 23
Columns: 125 entries, GISDATA_PR to geometry
dtypes: float64(11), geometry(1), int64(13), object(100)
memory usage: 23.6+ KB


### Format fields

In [8]:
df['PROJECT__6'] = df['PROJECT__6'].str.title()
df['PUBLIC_NAR'] = df['PUBLIC_NAR'].str.replace('(Sponor = City of Pittsburgh) ', '', regex=False)
df['PUBLIC_NAR'] = df['PUBLIC_NAR'].str.replace('(Sponsor = City of Pittsburgh)', '', regex=False)
df['PUBLIC_NAR'] = df['PUBLIC_NAR'].str.replace('(Project sponsor = City of Pittsburgh)', '', regex=False)

In [9]:
df['EST_CONSTR'] = df['EST_CONSTR'].map(lambda x: '${:,}'.format(int(x)))

In [10]:
df.loc[df['COMPLETI_1'].notna(), 'COMPLETI_1 DT'] = pd.to_datetime(df['COMPLETI_1'], format='%Y%m%d')

df.loc[df['COMPLETI_1'].notna(), 'COMPLETI_1'] = df['COMPLETI_1 DT'].dt.strftime('%m/%d/%Y')
df.loc[df['COMPLETI_1'].isna(), 'COMPLETI_1'] = 'Unknown'

In [11]:
df['NTP_DATE'] = pd.to_datetime(df['NTP_DATE'], format='%Y%m%d')
df['NTP_DATE'] = df['NTP_DATE'].dt.strftime('%m/%d/%Y')

In [12]:
df['lng'] = df.apply(lambda x: x['geometry'].coords[0][0], axis=1)
df['lat'] = df.apply(lambda x: x['geometry'].coords[0][1], axis=1)

### Datawrapper-ify fields

In [13]:
df['color'] = df['PROJECT__6']
df['color'] = df['color'].replace({ 'Future Development': 'black', 'In Development': 'yellow', 'Under Construction': 'orange' })

In [14]:
df['title'] = ' '
df['tooltip'] = df.agg(lambda x: f"<b>Construction start:</b> {x['NTP_DATE']}\n<b>Construction stop:</b> {x['COMPLETI_1']}\n<b>Construction cost:</b> {x['EST_CONSTR']}\n<b>Public narrative:</b> {x['PUBLIC_NAR']}\n", axis=1)

### Select columns to display

In [15]:
df[[
    'lng',
    'lat',
    'title',
    'color',
    'tooltip'
]]

,lng,lat,title,color,tooltip
0,-79.997600,40.394500,,black,<b>Construction start:</b> 07/14/2031\n<b>Cons...
1,-79.947500,40.457000,,black,<b>Construction start:</b> 08/14/2034\n<b>Cons...
2,-79.981200,40.465700,,black,<b>Construction start:</b> 06/21/2032\n<b>Cons...
3,-79.976800,40.423600,,black,<b>Construction start:</b> 06/14/2032\n<b>Cons...
4,-79.951818,40.454023,,black,<b>Construction start:</b> 06/14/2032\n<b>Cons...
5,-80.043300,40.451700,,black,<b>Construction start:</b> 09/11/2034\n<b>Cons...
6,-79.933500,40.456300,,yellow,<b>Construction start:</b> 09/13/2023\n<b>Cons...
7,-79.950500,40.426700,,yellow,<b>Construction start:</b> 03/09/2026\n<b>Cons...
8,-79.907600,40.467700,,yellow,<b>Construction start:</b> 08/26/2025\n<b>Cons...
9,-80.004900,40.472600,,yellow,<b>Construction start:</b> 12/07/2026\n<b>Cons...


In [16]:
df[[
    'lng',
    'lat',
    'title',
    'color',
    'tooltip'
]].to_csv('output/pgh-bridges.csv', index=False)